В этой практике вы познакомитесь с тем как строить пайплайн для решения задачи обучения с учителем. На самом деле строить пайплайн с нуля даже для такой задачи как классификация кошек и собак довольно трудоемко для первой практики, поэтому многие шаги я уже приготовил. Вам нужно реализовать ключевые куски чтобы сложить картину целиком.
Для того чтобы выполнить практику, поставьте [Anaconda](https://www.anaconda.com/download) - она содержит нужные для работы библиотеки.

Зарегистрируйтесь на платформе https://kaggle.com и посмотрите конкурс https://www.kaggle.com/c/dogs-vs-cats

Скачайте данные этого конкурса https://www.kaggle.com/c/3362/download-all. Наш проект будет построен по т.н. схеме [cookiecutter](https://drivendata.github.io/cookiecutter-data-science/#cookiecutter-data-science) (прочитать про неё полезно, но не обязательно), поэтому скачанные данные распакуйте и поместите в папку `data/` внутри проекта (на деле если вы используете ссылку выше нужно распаковать дважды - архив и архивы внутри архива). Если вы используете `git`, то добавьте папку `data/` в файл `.gitignore`.

Данные для каждой практики персональны для каждого студента. Введите свое имя, фамилию и группу в строковую переменную STUDENT_ID (в произвольной форме) - это ваш уникальный идентификатор.

In [30]:
STUDENT_ID = "<введите свое имя фамилию и группу здесь>"

Чтение данных
====

Обработаем данные. Этот шаг я сделаю за вас в силу некоторой его трудоемкости. Используем библиотеки pandas, numpy и Pillow.
- Введение в pandas и numpy можно посмотреть в первом уроке курса mlcourse_open: 
- https://github.com/Yorko/mlcourse.ai/blob/master/jupyter_russian/topic01_pandas_data_analysis/lesson1_part0_numpy.ipynb
- https://github.com/Yorko/mlcourse.ai/blob/master/jupyter_russian/topic01_pandas_data_analysis/lesson1_part1_pandas_intro.ipynb
- Pillow - одна из самых распространенных библиотек для работы с изображениями в python.

In [59]:
import pandas as pd
import pathlib
import hashlib
import numpy as np
import random


train_directory = pathlib.Path("data/train")
sample_size = 5000

def initialize_random_seed():
    """Инициализирует ГПСЧ из STUDENT_ID"""
    sha256 = hashlib.sha256()
    sha256.update(STUDENT_ID.encode("utf-8"))
    
    fingerprint = int(sha256.hexdigest(), 16) % (2**32) 
    
    random.seed(fingerprint)
    np.random.seed(fingerprint)


def read_target_variable():
    """Прочитаем разметку фотографий из названий файлов"""
    target_variable = {
        "filename": [],
        "is_cat": []
    }
    image_paths = list(train_directory.glob("*.jpg"))
    random.shuffle(image_paths)
    for image_path in image_paths[:sample_size]:
        filename = image_path.name
        class_name = filename.split(".")[0]
        target_variable["filename"].append(filename)
        target_variable["is_cat"].append(class_name == "cat")

    return pd.DataFrame(data=target_variable)


initialize_random_seed()

target_df = read_target_variable()
target_df

,filename,is_cat
0,dog.6665.jpg,False
1,dog.9541.jpg,False
2,cat.5469.jpg,True
3,dog.1612.jpg,False
4,cat.1795.jpg,True
5,dog.3625.jpg,False
6,dog.1945.jpg,False
7,dog.4209.jpg,False
8,cat.8388.jpg,True
9,dog.579.jpg,False


Обработка изображений
====

Используем изображения в качестве признакового описания. Признаками будут значения конкретных пикселей.

In [63]:
from PIL import Image
from tqdm import tqdm_notebook


def read_data(target_df):
    """Читает данные изображений и строит их признаковое описание"""
    image_size = (100, 100)
    features = []
    target = []
    for i, image_name, is_cat in tqdm_notebook(target_df.itertuples(), total=len(target_df)):
        image_path = str(train_directory / image_name)
        image = Image.open(image_path)
        image = image.resize(image_size) # уменьшаем изображения
        image = image.convert('LA') # преобразуем в Ч\Б
        pixels = np.asarray(image)[:, :, 0]
        pixels = pixels.flatten()
        features.append(pixels)
        target.append(is_cat)
    return np.array(features), np.array(target)

features, target = read_data(target_df)
features

array([[128, 152, 139, ..., 134, 159, 115],
       [ 89, 100,  94, ...,  67,  29,  26],
       [175, 177, 178, ..., 111, 122,  95],
       ...,
       [164, 164, 164, ..., 164, 164, 164],
       [ 51,  47,  47, ..., 128, 132, 125],
       [  7,  23,  16, ...,  57, 112, 140]], dtype=uint8)

Разбиение на обучающую, валидационную и тестовую выборки
====

Сделайте обучающую, валидационную и тестовую выборки в пропорции 60/20/20. Используйте библиотеку scikit-learn (метод train_test_split). 

Документация: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Советую изучить готовый пайплайн для другой задачи, чтобы примерно понимать что требуется делать дальше (в отличии от нашего пайплайна тут нет валидационного множества): https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html 


Обучение модели
====

Попробуйте обучить модель `sklearn.linear_model.SGDClassifier` на полученных данных. Если это занимает слишком много времени и ресурсов - уменьшайте размер выборки (переменная sample_size).

Документация: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

Настройка гиперпараметров
====

Подберите скорость обучения модели, оценивая долю верных ответов (scikit-learn accuracy_score) на валидационном множестве (подбираем параметр `eta0` в отрезке [0, 1] указав параметр `learning_rate="constant"`).

Оценка результатов
====

Оцените долю верных ответов модели (scikit-learn accuracy_score). Каков уровень переобучения?

Нормализация данных
====

Сделайте нормализацию данных с помощью StandardScaler из библиотеки sklearn (scaler.fit_transform на train и validation, scaler.transform на test). Оцените долю верных ответов модели на тестовой выборке. Какова ситуация теперь?

Документация: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

Сабмит на kaggle
====

(Опционально) сделайте сабмит в конкурсе https://www.kaggle.com/c/dogs-vs-cats (это простой бейзлайн, не ожидайте высокого результата)

# Random Forest

Примените RandomForest вместо SGDClassifier на этой задаче. Подберите количество деревьев, перебирая их в интервале от 100 до 400.


Оцените качество RandomForest на кросс-валидации по пяти фолдам. Посмотрите на среднее и стандартное отклонение точности на кросс-валидации.